<a href="https://colab.research.google.com/github/shreyas-desai-stevens/KDDM-project/blob/main/KDDM_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import  sklearn as sk
import pandas as pd